In [ ]:
file_args_key = "shake"  
# file_args_key = "live"

proof_number = 0 # 0, 1,2, 3, 6
keynumber0816 = 81 # 1、4、80、81

# yesorno_inputdim = 100
yesorno_inputdim = 50  

kkkkkkkkkkkk = 0   # 0、1、2、3、4

key_read_write = 'write'  
# key_read_write = 'read'  

# raise_Exception = True  
raise_Exception = False 

batch_sanpling_bur = True 
# batch_sanpling_bur = False 


In [ ]:
# import nltk
# nltk.download('averaged_perceptron_tagger')

没有小批次的实验

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torchtext.legacy import data, datasets

from sklearn.model_selection import KFold
import copy
# from tqdm import tqdm
import numpy as np
import random
from model_nn import RNN, RNN50
# from model_nn_min import RNN, RNN_gradient,RNN50, RNN50_gradient
from args_fedavg import args_parser_live, args_parser_shake

from nltk.tag import pos_tag
import re
import time


In [ ]:
out_log = r"/mnt/workspace/fedavg_ceshi/out_log"
out_pth = r"/mnt/workspace/fedavg_ceshi/out_pth"
write_read_txt = r"/mnt/workspace/fedavg_ceshi/write_read_txt"

if file_args_key == "shake":
    args = args_parser_shake()
elif file_args_key == "live":
    args = args_parser_live()

# batchsz = args.B
# sampling_rate = args.C
# cpu or GPU
# device = args.device
# clients_wind = args.clients

String = time.strftime("%m_%d_%Y_%H_%M_%S",time.localtime())
namestr = str(String)

file_write_log_name = out_log + "/" + "log_" + file_args_key + '_'  + str(proof_number) \
    + '_'+ str(keynumber0816) + '_'+ str(kkkkkkkkkkkk) + '_' \
        + str(args.B) + '_'+ str(args.C) + '_' + key_read_write + '_'+ namestr + "_log.txt"
print(file_write_log_name)

file_write_log = open(file_write_log_name, "w", encoding='utf-8')

save_name_pth = out_pth + "/" +  file_args_key + '_'  + str(proof_number) \
    + '_'+ str(keynumber0816) + '_'+ str(kkkkkkkkkkkk) + '_' \
        + str(args.B) + '_'+ str(args.C) + '_' + key_read_write + '_'+ namestr +'_dict.pth'
print(save_name_pth)

writecon = str(save_name_pth) + "\n"
file_write_log.write(writecon)

In [ ]:
TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

if file_args_key == "shake":
    # train_data.examples = train_data.examples[:len(train_data.examples)//500]
    # test_data.examples = test_data.examples[:len(test_data.examples)//500]

    train_data.examples = train_data.examples[:len(train_data.examples)//125]
    test_data.examples = test_data.examples[:len(test_data.examples)//125]

len_train_data = len(train_data)
len_test_data = len(test_data)

print('len of train data:', len(train_data))
print('len of test data:', len(test_data))
print(test_data.examples[1].text) 
print(test_data.examples[1].label)


In [ ]:
# word2vec, glove
if yesorno_inputdim == 100:
    TEXT.build_vocab(train_data, max_size=10000, vectors='glove.6B.100d')
if yesorno_inputdim == 50:
    TEXT.build_vocab(train_data, max_size=10000, vectors='glove.6B.50d')

LABEL.build_vocab(train_data)

# TEXT.build_vocab(train_data, max_size=10000, vectors='glove.6B.50d')
# LABEL.build_vocab(train_data)

# TEXT.build_vocab(test_data, max_size=10000, vectors='glove.6B.100d')
# LABEL.build_vocab(test_data)

vocab_size =  len(TEXT.vocab)
pretrained_embedding = TEXT.vocab.vectors
print(vocab_size, pretrained_embedding.shape)

In [ ]:
# proof1、proof3、proof6 
def RFFM(i, x):
    torch.manual_seed(i)
    input_dim = 100
    output_dim = 50
    sigma = 1.0
    omega = torch.randn(output_dim, input_dim) * sigma
    b = torch.rand(output_dim) * 2 * np.pi
    x = x.view(-1, input_dim)
    z = torch.cos(torch.mm(x, omega.t()) + b)
    Z_Out =  torch.sqrt(2.0 / torch.tensor(output_dim)) * z
    return Z_Out
    # return z

In [ ]:
# proof1、proof3、proof6 
if yesorno_inputdim == 100:
    new_vectors = []
    for i, word in enumerate(TEXT.vocab.itos):
        old_vector = pretrained_embedding[i]
        new_vector = RFFM(i, old_vector)
        new_vectors.append(new_vector)
    new_pretrained_embedding = torch.stack(new_vectors, dim=0)
    new_pretrained_embedding = new_pretrained_embedding.reshape(-1,50)
    print(new_pretrained_embedding.shape)
elif yesorno_inputdim == 50:
    pass

# torch.cuda.empty_cache()

In [ ]:
# proof1、proof3、proof6 
def word_vector_50(word):
    # word = 'in'
    word_index = TEXT.vocab.stoi[word]
    # print(word_index)
    word_vector = new_pretrained_embedding[word_index]
    # print(word_vector)
    return word_vector

def proof1(a, b, epcluo):
    a = a.to(torch.float)
    b = b.to(torch.float)
    dot_product = torch.dot(a, b)
    norm_a = torch.norm(a, p=2)
    norm_b = torch.norm(b, p=2)
    similarity = dot_product / (norm_a * norm_b)
    return torch.exp(similarity* epcluo / 2.0 *(-1))


def proof1_change(word_list,  proper_nouns, filtered, proper_nouns_pos):
    if len(proper_nouns) == 0 or len(filtered) == 0 :  #proper_nouns, filtered 任一为空
        return word_list
    # epcluo = args.epcluo
    epcluo = args.epcluo_05
    gam = []
    xlist = [word_vector_50(word) for word in proper_nouns]
    ylist = [word_vector_50(word) for word in filtered]
    
    for x in xlist:
        x_list = []
        for y in ylist:
            res = proof1(x, y, epcluo)
            x_list.append(res.item())
        gam.append(x_list)
    gam_tensor = torch.tensor(gam)
    # print(gam_tensor)
    row_sums = torch.sum(gam_tensor, dim=1, keepdim=True)  
    result_prob = torch.div(gam_tensor, row_sums) 

    # probabilities = result_prob.clone()
    # probabilities = result_prob
    sample = torch.multinomial(result_prob, 1)
    # print(sample)
    # tensor = sample.clone()
    # tensor = sample
    list_result = sample.squeeze().tolist()
    # print(list_result)
    # print(filtered)
    try:
        list_result = list(list_result)
    except:
        list_result = [list_result]

    rresul = [filtered[i] for i in list_result]
    # print('-------------')
    # print('rresul',longg, rresul)
    iiindex = 0
    for index, word in enumerate(proper_nouns_pos):
        # print('index', index,  word)
        if (word[0] in proper_nouns) and (word[1].startswith('NNP') or word[1].startswith('NNPS') ):
            # print(iiindex)
            newword = rresul[iiindex] 
            word_list[index] = newword
            iiindex += 1   
    return word_list


In [ ]:
def word_vector_100(word):
    # word = 'in'
    word_index = TEXT.vocab.stoi[word]
    # print(word_index)
    word_vector = TEXT.vocab.vectors[word_index]
    # print(word_vector)
    return word_vector

def proof2(x, y, epcluo):
    K = torch.exp(torch.norm(x-y, p=2).pow(2) / 2.0 *(-1) )
    result = K * epcluo / 2.0 *(-1) 
    # print('result: ', result)
    e14 = torch.exp(result)
    # print(e14)
    return e14


def proof2_change(word_list,  proper_nouns, filtered, proper_nouns_pos):
    if len(proper_nouns) == 0 or len(filtered) == 0 :  #proper_nouns, filtered 任一为空
        return word_list
    # epcluo = args.epcluo
    epcluo = args.epcluo_05
    gam = []
    xlist = [word_vector_100(word) for word in proper_nouns]
    ylist = [word_vector_100(word) for word in filtered]
    
    for x in xlist:
        x_list = []
        for y in ylist:
            res = proof2(x, y, epcluo)
            x_list.append(res.item())
        gam.append(x_list)
    gam_tensor = torch.tensor(gam)
    # print(gam_tensor)
    row_sums = torch.sum(gam_tensor, dim=1, keepdim=True)  
    result_prob = torch.div(gam_tensor, row_sums)  
    # probabilities = result_prob
    sample = torch.multinomial(result_prob, 1)
    # print(sample)
    # tensor = sample
    list_result = sample.squeeze().tolist()

    # print(list_result)
    # print(filtered)

    try:
        list_result = list(list_result)
    except:
        list_result = [list_result]
    rresul = [filtered[i] for i in list_result]
    # longg = len(rresul)
    # print('-------------')
    # print('rresul',longg, rresul)
    iiindex = 0
    for index, word in enumerate(proper_nouns_pos):
        # print('index', index,  word)
        # if (word in proper_nouns) and (iiindex < longg):
        if (word[0] in proper_nouns) and (word[1].startswith('NNP') or word[1].startswith('NNPS') ):
            # print(iiindex)
            newword = rresul[iiindex]
            word_list[index] = newword
            iiindex += 1
    return word_list

In [ ]:
def proof3(zx, zy, epcluo): 
    result = (torch.dot(zx, zy)) * epcluo / 4.0 *(-1) 
    # print('result: ', result)
    e14 = torch.exp(result)
    # print(e14)
    return e14

def proof3_change(word_list,  proper_nouns, filtered, proper_nouns_pos):
    if len(proper_nouns) == 0 or len(filtered) == 0 :  
        return word_list
    # epcluo = args.epcluo
    epcluo = args.epcluo_05
    gam = []
    xlist = [word_vector_50(word) for word in proper_nouns]
    ylist = [word_vector_50(word) for word in filtered]

    for x in xlist:
        x_list = []
        for y in ylist:
            res = proof3(x, y, epcluo)
            x_list.append(res.item())
        gam.append(x_list)
    gam_tensor = torch.tensor(gam)
    # print(gam_tensor)
    row_sums = torch.sum(gam_tensor, dim=1, keepdim=True)  
    result_prob = torch.div(gam_tensor, row_sums)  
    # print(result_prob)
    # probabilities = result_prob
    sample = torch.multinomial(result_prob, 1)
    # print(sample)
    # tensor = sample
    list_result = sample.squeeze().tolist()

    # print(list_result)
    # print(filtered)
    
    try:
        list_result = list(list_result)
    except:
        list_result = [list_result]
    rresul = [filtered[i] for i in list_result]
    # longg = len(rresul)
    # print('-------------')
    # print('rresul',longg, rresul)
    iiindex = 0
    # for index, word in enumerate(word_list):
    for index, word in enumerate(proper_nouns_pos):
        # print('index', index,  word)
        # if (word in proper_nouns) and (iiindex < longg):
        if (word[0] in proper_nouns) and (word[1].startswith('NNP') or word[1].startswith('NNPS') ):
            # print(iiindex)
            newword = rresul[iiindex]
            word_list[index] = newword
            iiindex += 1
    return word_list

In [ ]:
def proof6(zx, zy, epcluo):

    result = (torch.dot(zx, zy)) * epcluo / 4.0 *(-1) 
    # print('result: ', result)
    e14 = torch.exp(result)
    # print(e14)
    return e14

# def proof6_change(word_list,  proper_nouns, filtered, proper_nouns_pos):
def proof6_change(word_list,  proper_nouns, filtered, proper_nouns_pos, filtered_y, proper_nouns_x ):
    if len(proper_nouns) == 0 or len(filtered) == 0 :  
        return word_list
    # epcluo = args.epcluo
    epcluo = args.epcluo_05
    gam = []
    xlist = [word_vector_50(word) for word in proper_nouns]
    ylist = [word_vector_50(word) for word in filtered]
    ylist_f = [word_vector_50(word) for word in filtered_y]
    xlist_f = [word_vector_50(word) for word in proper_nouns_x]

    for x in xlist:
        x_list = []
        for y in ylist:
            res = proof6(x, y, epcluo)
            x_list.append(res.item())
        gam.append(x_list)

    gam_tensor = torch.tensor(gam)
    # print(gam_tensor)
    row_sums = torch.sum(gam_tensor, dim=1, keepdim=True)  
    result_prob = torch.div(gam_tensor, row_sums)  
    # print(result_prob)
    probabilities = result_prob.clone()
    sample = torch.multinomial(probabilities, 1)
    # print(sample)
    tensor = sample.clone()
    list_result = tensor.squeeze().tolist()
    # print(list_result)
    # print(filtered)
    try:
        list_result = list(list_result)
    except:
        list_result = [list_result]

    rresul = [filtered[i] for i in list_result]
    # longg = len(rresul)
    # print('-------------')
    # print('rresul',longg, rresul)
    pr = args.pr
    gam_pr = []
    for y in ylist_f:
        x_list_new = []
        for x in xlist_f:
            res = proof6(y, x, epcluo)
            x_list_new.append(res.item())
        gam_pr.append(x_list_new)

    gam_pr_tensor = torch.tensor(gam_pr)
    row_pr_sums = torch.sum(gam_pr_tensor, dim=1, keepdim=True)  
    result_prob = pr * torch.div(gam_pr_tensor, row_pr_sums)  
    column_to_add = torch.full((result_prob.shape[0], 1), pr)
    tensor_with_column = torch.cat((result_prob, column_to_add), dim=1)
    # sampling
    probabilities = tensor_with_column.clone()
    sample_pr = torch.multinomial(probabilities, 1)
    # print(sample)
    tensor_pr = sample_pr.clone()
    list_result_pr = tensor_pr.squeeze().tolist()

    try:
        list_result_pr = list(list_result_pr)
    except:
        list_result_pr = [list_result_pr]

    rresuly = []
    xylong = len(tensor_with_column[0,:]) - 1

    for index, yword in enumerate(list_result_pr):
        if yword == xylong :
            rresuly.append(filtered_y[index])
        else:
            rresuly.append(proper_nouns_x[yword])
    # longgy = len(rresuly)
    # print('-------------')
    # print('rresul',longg, rresul)
    iiindex = 0
    iiindey = 0
    for index, word in enumerate(proper_nouns_pos):
        # print('index', index,  word)
        # if (word in proper_nouns) and (iiindex < longg):
        if (word[0] in proper_nouns) and (word[1].startswith('NNP') or word[1].startswith('NNPS') ):
            # print(iiindex)
            newword = rresul[iiindex]
            word_list[index] = newword
            iiindex += 1

        elif (word[0] in filtered_y) and not (word[1].startswith('NNP') or word[1].startswith('NNPS')) :  
        # else:
            # print(iiindey)
            newword = rresuly[iiindey]
            word_list[index] = newword
            iiindey += 1  
    return word_list

In [ ]:
def filter_proper_nouns(words):
    tagged_words = pos_tag(words)
    proper_nouns = []
    filtered_y = []
    proper_nouns_pos = []
    for word, pos in tagged_words:
        if pos.startswith('NNP') or pos.startswith('NNPS'):
            proper_nouns.append(word)
        else:
            filtered_y.append(word)
        proper_nouns_pos.append((word,pos))
    return proper_nouns, proper_nouns_pos, filtered_y


def zxy_split(word_list, proof_number):
    proper_nouns, proper_nouns_pos, filtered_y = filter_proper_nouns(word_list)
    # print(len(proper_nouns), proper_nouns)
    filtered = list(set(word_list) - set(proper_nouns))
    # print(filtered)
    if  proof_number == 1:
        new_word_list = proof1_change(word_list,  proper_nouns, filtered, proper_nouns_pos)

    elif proof_number == 2:
        new_word_list = proof2_change(word_list,  proper_nouns, filtered, proper_nouns_pos)

    elif proof_number == 3:
        new_word_list = proof3_change(word_list,  proper_nouns, filtered, proper_nouns_pos)

    elif proof_number == 6:
        proper_nouns_x = list(set(proper_nouns))
        # proof6_change(word_list,  proper_nouns, filtered, proper_nouns_pos)
        new_word_list = proof6_change(word_list,  proper_nouns, filtered, proper_nouns_pos, filtered_y, proper_nouns_x)
    elif proof_number == 0:    
        new_word_list = word_list
    else:
        pass
    # print(new_word_list)
    return new_word_list


In [ ]:
def geshj_tra_tes(data, train_or_testa):
    for index in range(len(data)):
        if ((index+1) % 1000 == 0):
            print(index)
        # print(index)
        # if index > 10:
        #     break

        # print('------------------------------')
        # print(test_data.examples[index].text, test_data.examples[index].label)
        atexta = data.examples[index].text
        filtered_aaa0 = [word for word in atexta if re.match(r'^[A-Za-z.,!?0-9]+$', word)]
        # print('---------------------:', filtered_aaa0)
        if train_or_testa == "oftrain":
            word_listaaa = zxy_split(filtered_aaa0, proof_number)
            data.examples[index].text = word_listaaa
        elif train_or_testa == "oftesta":
            data.examples[index].text = filtered_aaa0
        
        # print('>>>>>>')
        # print(test_data.examples[index].text)

In [ ]:
def chulia(astring):
    astring = astring.strip('\n').split('#')
    astring = [i for i in astring if len(i)>0]
    return astring
# torch.cuda.empty_cache()

def train_test_read(tr_te, data):
    for index in range(len(tr_te)):       
        word_listaaa = data[index]
        tr_te.examples[index].text = word_listaaa

        # if index % 1000 == 0:
        #     print(index)
        if (file_args_key == "live") and (index % 1000 == 0):
            print(index)
        if (file_args_key == "shake") and (index % args.B == 0):
            print(index)
        # print('>>>>>>')
        # print(test_data.examples[index].text)
# torch.cuda.empty_cache()

def read_or_write(key, proof_number):
    
    # file_args_key = "live"
    # file_args_key = "shake"
    if proof_number == 1:
        if file_args_key == "live":
            file_name_train = 'train_data_change_result_proof1_Zout_sig.txt'
            file_name_test  = 'test_data_change_result_proof1_Zout_sig.txt'
        elif file_args_key == "shake":
            file_name_train = 'train_proof1_50_Zout_sig.txt'
            file_name_test = 'test__proof1_50_Zout_sig.txt'

    if proof_number == 2:
        if file_args_key == "live":
            file_name_train = 'train_data_change_result_proof2_Zout_sig.txt'
            file_name_test = 'test_data_change_result_proof2_Zout_sig.txt'
        elif file_args_key == "shake":
            file_name_train = 'train_proof2_50_Zout_sig.txt'
            file_name_test = 'test__proof2_50_Zout_sig.txt'

    if proof_number == 3:
        if file_args_key == "live":
            file_name_train = 'train_data_change_result_proof3_Zout_sig.txt'
            file_name_test = 'test_data_change_result_proof3_Zout_sig.txt'
        elif file_args_key == "shake":
            file_name_train = 'train_proof3_50_Zout_sig.txt'
            file_name_test = 'test__proof3_50_Zout_sig.txt'

    if proof_number == 6:
        if file_args_key == "live":
            file_name_train = 'train_data_change_result_proof6_Zout_sig.txt'
            file_name_test  = 'test_data_change_result_proof6_Zout_sig.txt'
        elif file_args_key == "shake":
            file_name_train = 'train_proof6_50_Zout_sig.txt'
            file_name_test = 'test__proof6_50_Zout_sig.txt'
    
    if proof_number == 0:
        if file_args_key == "live":
            file_name_train = 'train_data_change_result_proof0_Zout_sig.txt'
            file_name_test  = 'test_data_change_result_proof0_Zout_sig.txt'
        elif file_args_key == "shake":
            file_name_train = 'train_proof0_50_Zout_sig.txt'
            file_name_test = 'test__proof0_50_Zout_sig.txt'

    file_name_train = write_read_txt + "/" + file_name_train
    file_name_test = write_read_txt + "/" + file_name_test

    if key=='read':
        fead_train = open(file_name_train, 'r', encoding='utf-8').readlines()
        # fead_train = fead_train[:len(fead_train)//2]
        fead_test = open(file_name_test, 'r', encoding='utf-8').readlines()
        # fead_test = fead_test[:len(fead_test)//2]
        # print(fead_test[:10])
        # train_read(fead_train)
        # test_read(fead_test)

        train_test_read(train_data, fead_train)
        train_test_read(test_data, fead_test)

    elif key=='write':      
        geshj_tra_tes(train_data, "oftrain")
        file_write(file_name_train, len_train_data, train_data)

        geshj_tra_tes(test_data, "oftesta")
        file_write(file_name_test, len_test_data, test_data)
    else:
        pass

def list_to_string(alist):
    string = ''
    for i in alist:
        string += i + '#'
    return string

def file_write(filename, len_data, data):
    f = open(filename, 'w', encoding = 'utf-8')
    for index in range(len_data):
        strings = list_to_string(data.examples[index].text)
        f.write(strings + '\n')
    f.close()



In [ ]:
read_or_write(key_read_write, proof_number)

if raise_Exception and (key_read_write == 'write') :
    raise Exception("Terminated at this point")


In [ ]:
train_indices = list(range(len(train_data.examples)))

k = args.K 
random_seed = 42  
kfold = KFold(n_splits=k, shuffle=True, random_state=random_seed)

train_subsets = []
for _, train_idx in kfold.split(train_indices):
    train_subset = data.Dataset([train_data.examples[i] for i in train_idx], fields=[('text', TEXT), ('label', LABEL)])
    # print(len(train_subset))
    train_subsets.append(train_subset)

In [ ]:
test_indices = list(range(len(test_data.examples)))

random_seed = 24  
kfold_te = KFold(n_splits=k, shuffle=True, random_state=random_seed)

test_subsets = []
for _, test_idx in kfold_te.split(test_indices):
    test_subset = data.Dataset([test_data.examples[i] for i in test_idx], fields=[('text', TEXT), ('label', LABEL)])
    # print(len(test_subset))
    test_subsets.append(test_subset)

In [ ]:
def iterator(tri_data, tes_data):
    
    train_iterator, test_iterator = data.BucketIterator.splits(
        (tri_data, tes_data),
        batch_size = args.B,
        sort_key = lambda x: len(x.text),
        sort_within_batch=True,
        # device=torch.device("cpu") 
        device=args.device 
    )

    return train_iterator, test_iterator

In [ ]:
train_all_sub = []
testa_all_sub = []

for index in range(len(train_subsets)) :
    data_i, data_j = train_subsets[index ], test_subsets[index ]
    data_i_reu, data_j_reu = iterator(data_i, data_j)
    train_all_sub.append(data_i_reu)
    testa_all_sub.append(data_j_reu)


### 下面就是之前的单个文件

In [ ]:
batchsz = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clients_wind = ['Task' + str(i) for i in range(0, 20)]

In [ ]:
loss_function = nn.BCEWithLogitsLoss().to(args.device) 

def get_val_loss(model, Vall):  
    model.eval()
    val_loss = []
    with torch.no_grad():
        for seqq in Vall:
            y_preda = model(seqq.text).squeeze(1)
            loss = loss_function(y_preda, seqq.label)
            val_loss.append(loss.item())
    loss_v = np.array(val_loss).mean()
    # print('get_val_loss ----- val_loss',val_loss)
    # print('get_val_loss ----- np.array(val_loss).mean()', loss_v)
    return loss_v 


def binary_acc(preds, y):
    """
    get accuracy
    """
    preds = torch.round(torch.sigmoid(preds))
    correct = torch.eq(preds, y).float()
    acc = correct.sum() / len(correct)
    return acc


def train(args, model, k):  
    model.train()
    # train_all_sub
    # testa_all_sub
    Dtr = train_all_sub[k]   
    Val = testa_all_sub[k]
    model.Len = len(Dtr) 
    # print('model.name', model.name)
    # print('model.len:', model.Len)

    lr = args.lr
    if args.optimizer == 'adam': 
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3,
                                     weight_decay=args.weight_decay)  
    else:
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3,
                                    momentum=0.9, weight_decay=args.weight_decay)
    
    # lr_step = StepLR(optimizer, step_size=args.step_size, gamma=args.gamma) 
    # training
    min_epochs = 1
    best_model = None
    # min_val_loss = 10
    # min_loss_list = [100]
    min_val_loss = 10000.0

    for epoch in range(args.E):  
        train_loss = [] 
        for seq in Dtr:
            # print(seq)
            # seq = seq.to(device) 
            y_pred = model(seq.text).squeeze(1)
            loss = loss_function(y_pred, seq.label)
            train_loss.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    # lr_step.step()

        val_loss = get_val_loss(model, Val)
        if (file_args_key == "shake") and ((epoch+1) % 1 == 0) :
            # print('epoch {:03d} 训练 train_loss {:.8f} 验证 val_loss {:.8f}'.format(epoch, np.array(train_loss).mean(), val_loss))
            wstring = f"epoch {epoch} 训练 train_loss {np.array(train_loss).mean()} 验证 val_loss {val_loss}"
            writecon = str(wstring) + "\n"
            file_write_log.write(writecon)

        if (file_args_key == "live") and ((epoch+1) % 1 == 0):
            # print('epoch {:03d} 训练 train_loss {:.8f} 验证 val_loss {:.8f}'.format(epoch, np.array(train_loss).mean(), val_loss))
            wstring = f"epoch {epoch} 训练 train_loss {np.array(train_loss).mean()} 验证 val_loss {val_loss}"
            writecon = str(wstring) + "\n"
            file_write_log.write(writecon)

        if (epoch + 1 >= min_epochs) and (val_loss <= min_val_loss) : # 达到最小迭代次数（10），并且损失最优（5）
            min_val_loss = val_loss
        # validation
        model.train()

    avg_val_lossa = np.array(train_loss).mean()
    wstring = f"model {k} val_loss: {avg_val_lossa} "
    writecon = str(wstring) + "\n"
    file_write_log.write(writecon)
    # print('epoch {:03d} train_loss {:.8f} val_loss ...'.format(epoch, np.array(train_loss).mean()))
    wstring = f"训练 best min train_loss {min_val_loss} "
    writecon = str(wstring) + "\n"
    file_write_log.write(writecon)
    print('model {}, avg_loss {:.8f},  min_loss {:.8f}, val loss {:.8f} '.format(k, avg_val_lossa,min_val_loss, val_loss ))
        # if (epoch+1) % 5 == 0:
        #     print('epoch {:03d} 训练 train_loss {:.8f} 验证 val_loss {:.8f}'.format(epoch, np.array(train_loss).mean(), val_loss))
    # return best_model
    return model

def test(args, model, k):
    model.eval()
    Val = testa_all_sub[k] 
    # pred = []
    # y = []
    avg_acc = []
    # device = device
    with torch.no_grad():
        for seq in Val:
            # seq = seq.to(device)  
            y_pred = model(seq.text).squeeze(1) 
            loss = loss_function(y_pred, seq.label)
            acc = binary_acc(y_pred, seq.label).item()
            avg_acc.append(acc)
    avg_acc = np.mean(avg_acc)
    print("客户端：", k, ' avg acc', avg_acc)
    wstring = f"客户端：{k} avg acc: {avg_acc}"
    writecon = str(wstring) + "\n"
    file_write_log.write(writecon)



In [ ]:
class FedAvg: 

    def __init__(self, args):
        self.args = args 
        self.clients = args.clients  # ['Task_' + str(i) for i in range(0, 20)]

        # self.nn = RNN(vocab_size, args, name='server').to(device)   
        # self.nn = RFFMRNN(vocab_size, args, name='server').to(device)  
        if yesorno_inputdim == 50:
            self.nn = RNN50(vocab_size, args, name='server').to(args.device)   
            self.nn.embedding.weight.data.copy_(pretrained_embedding)
        elif (proof_number == 1) or (proof_number == 3) or (proof_number == 6): 
            self.nn = RNN50(vocab_size, args, name='server').to(args.device)   
            self.nn.embedding.weight.data.copy_(new_pretrained_embedding)
        elif (proof_number == 0) or (proof_number == 2) :
            self.nn = RNN(vocab_size, args, name='server').to(args.device)  
            self.nn.embedding.weight.data.copy_(pretrained_embedding)

        self.nns = []

        for i in range(self.args.K):    
            temp = copy.deepcopy(self.nn)
            temp.name = self.clients[i]  
            self.nns.append(temp)  
    


    def server(self):
        for t in range(self.args.r):  
            # file.write('round-------------!!!!!!!!!!!!!!!!!!!!!!!!!' + str(t + 1) + ':' + '\n')
            print('round：--------', t + 1, '------------------------')
            wstring = f"round：-------- {t + 1} ------------------------"
            writecon = str(wstring) + "\n"
            file_write_log.write(writecon)
            m = np.max([int(self.args.C * self.args.K), 1]) 
            index = random.sample(range(0, self.args.K), m)  
            print('选取客户端:',index)
            wstring = f"选取客户端: {index} "
            writecon = str(wstring) + "\n"
            file_write_log.write(writecon)
            # dispatch
            self.dispatch(index) 
            # local updating
            self.client_update(index)  
            # aggregation
            self.aggregation(index)  

        torch.save(self.nn.state_dict(), save_name_pth)
        print(save_name_pth, 'save')
        return self.nn

    def dispatch(self, index): 
        for j in index:
            for old_params, new_params in zip(self.nns[j].parameters(), self.nn.parameters()):
                old_params.data = new_params.data.clone()

    def client_update(self, index):  
        for k in index: 
            wstring = f"训练本地模型： {k} "
            writecon = str(wstring) + "\n"
            file_write_log.write(writecon)
            self.nns[k] = train(self.args, self.nns[k], k)  

    def aggregation(self, index):  # 聚合方法
        s = 0
        for j in index:
            net = self.nns[j]
            s += net.Len
        params = {}
        for k, v in self.nns[0].named_parameters(): 
            params[k] = torch.zeros_like(v.data)

        for j in index:
            for k, v in self.nns[j].named_parameters(): 
                params[k] += v.data * (self.nns[j].Len / s)  

        for k, v in self.nn.named_parameters():
            v.data = params[k].data.clone() 


    def global_test(self, args): 
        model = self.nn 
        model.eval()  
        c = clients_wind # ['Task' + str(i) for i in range(0, 20)]
        for k, client in enumerate(c):
            model.name = client  
            # result_acc = test(self.args, model, k)
            test(self.args, model, k)
            # f.write(str(result_acc) + '\n')


In [ ]:
fed = FedAvg(args)

fed.server()

In [ ]:
fed.global_test(args)

In [ ]:
file_write_log.close()